In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
from datagen_pytorch import DataGenerator
from models import Resnet
from torchsummary import summary
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
from torchvision.transforms import ToPILImage
import numpy as np
from og_loss import lc_loss
from utils_pytorch import visualize_dots
from skimage import morphology as morph

In [5]:
gen = DataGenerator(root='__misc/', augment=False, scale=5, split='train', debug_mode=True)

In [ ]:
batch = gen.__getitem__(0)
print(np.asarray(batch['points']).shape)
print(gen.__len__())


In [ ]:
visualize_dots(np.asarray(batch['OG_image']), np.asarray(batch['points']))

In [ ]:
dataloader = DataLoader(gen)

In [ ]:
model = Resnet(n_classes=2, layers=101).cuda()

In [ ]:
torch.manual_seed(1) 
torch.cuda.manual_seed_all(1)


In [ ]:
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5, weight_decay=0.0005)

In [ ]:
for epoch in range(100):
    loss_sum = 0.
    for batch in dataloader:
        opt.zero_grad()
        loss = lc_loss(model, batch)
        loss.backward()
        opt.step()
        
        loss_sum+=loss.item()
        
    print("\nepoch:{} loss:{}".format(epoch+1, loss_sum/len(dataloader)))
    out = model(batch['images'].cuda())
    out = torch.nn.functional.softmax(out,1)
    out_np = out.cpu().detach().numpy()[0]

    img = np.asarray(batch['OG_image'][0])
    visualize_dots(img, (out_np[1] > 0.5).astype(int), save=True, name=epoch)

In [ ]:
out = model(batch['images'].cuda())

In [ ]:
out = torch.nn.functional.softmax(out,1)

In [ ]:
out_np = out.cpu().detach().numpy()[0]

In [ ]:
img = np.asarray(batch['OG_image'][0])

In [ ]:
visualize_dots(img, (out_np[1] > 0.5).astype(int), save=False)

In [3]:
gen = DataGenerator(root='__misc/', scale=5, split='test', augment=False)
batch = gen.__getitem__(0)


In [4]:
batch['images'].shape

torch.Size([3, 800, 1200])

In [5]:
dataloader = DataLoader(gen)

In [6]:
for i,b in enumerate(dataloader):
    print(b['images'].shape)
    break



torch.Size([1, 3, 800, 1200])


In [7]:
model = Resnet(n_classes=2, layers=101).cuda()

In [8]:
model.load_state_dict(torch.load('model_data/model/570.pth'))

In [9]:
model.eval()

for batch in dataloader:
    out = model(batch['images'].cuda())
    



In [10]:
out_np = (torch.nn.functional.softmax(out,1).cpu().detach().numpy()[0][1] > 0.9).astype(int)

In [22]:
img = np.asarray(batch['OG_image'][0])

In [12]:
visualize_dots(img, out_np, save=False, size=5)

In [13]:
labelled, num = morph.label(out_np, return_num=True)

In [17]:
img.shape[1]

1200

In [23]:
img = cv2.putText(img, str(morph.label(out_np, return_num=True)[1]), (0,0), cv2.FONT_HERSHEY_SIMPLEX, 4, (255,255,255))
cv2.imshow("Dots Visualized", img)
cv2.waitKey(0) 
cv2.destroyWindow("Dots Visualized")